In [1]:
cd ..

d:\research\fedmiss_bench


In [2]:
import os
import itertools
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

nnn_imp_fed = {
    'simple': ['local', 'fedavg'],
    'em': ['local', 'fedavg'],
    'linear_ice': ['local', 'fedavg'],
    'missforest': ['local', 'fedtree']
}

nn_imp_fed = {
    'miwae': ['local', 'fedavg', 'fedavg_ft', 'fedprox'],
    'gain': ['local', 'fedavg', 'fedavg_ft', 'fedprox'],
}

dataset_dp = {
    'vehicle_np': ['natural-partition'],
    'school_np': ['natural-partition'],
    'hhip_np': ['natural-partition'],
}

dataset_metric_pred = {
    'vehicle_np': ['accuracy', 'f1', 'auc', 'prc'],
    'school_np': ['mse', 'mae', 'r2', 'msle'],
    'hhip_np': ['mse', 'mae', 'r2', 'msle'],
}

In [6]:
def load_ret(datasets, imputers, met):
    
    rets = []
    unfinished = {}
    for dataset in datasets:
        
        data_partition = dataset_dp[dataset]
        unfinished[dataset] = {}
        missing = ['mcar', 'mar-homog', 'mnar2-homog', 'mar-heter', 'mnar2-heter']
        models = ['nn']
        
        for imputer in imputers:
            unfinished[dataset][imputer] = set()
            print(dataset, imputer)            
            if imputer in nnn_imp_fed:
                rounds = [0,1,2,3,4,5,6,7,8,9]
                feds = nnn_imp_fed[imputer]
            else:
                rounds = [0,1,2,3,4]
                feds = nn_imp_fed[imputer]

            param = itertools.product(feds, data_partition, missing, rounds, models)
            dir_path = f'./results/raw_results/eval_results/alpha/{dataset}/'

            count, total = 0, 0
            for i, (fed, dp, ms, round, model) in enumerate(param):
                
                file_path = f'{dir_path}{dp}_{ms}/{imputer}/{fed}/{round}/{model}/eval_results.json'
                total += 1
                try:
                    with open(file_path, 'r') as f:
                        ret = json.load(f)
                        if met == 'imp':
                            metrics = ['rmse', 'sliced-ws']
                            ret_metrics = [ret['results']['imp_quality'][metric] for metric in metrics]
                        elif met == 'pred':
                            metrics = dataset_metric_pred[dataset]
                            ret_metrics = [ret['results']['pred_performance'][metric] for metric in metrics]
                        else:
                            raise ValueError('Invalid metric')
                        
                        for id, items in enumerate(zip(*ret_metrics)):
                            setting = {
                                'dataset': dataset,
                                'data_partition': dp,
                                'imputer': imputer,
                                'fed': fed,
                                'missing': ms,
                                'round': round,
                                'model': model,
                                'client_id': id
                            }
                            setting.update({metric: item for metric, item in zip(metrics, items)})
                            rets.append(setting)
                
                except Exception as e:
                    print(e)
                    count += 1
                    unfinished[dataset][imputer].add((dataset, dp, ms, imputer, fed, round, model, e))
                    continue

            print(f'{count}/{total}')
        
    return rets, unfinished

In [7]:
result_pred, unfinished = load_ret(
    ['vehicle_np', 'school_np', 'hhip_np'],
    ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain'], 
    met = 'pred'
)

vehicle_np simple
0/100
vehicle_np em
0/100
vehicle_np linear_ice
0/100
vehicle_np missforest
0/100
vehicle_np miwae
0/100
vehicle_np gain
0/100
school_np simple
0/100
school_np em
0/100
school_np linear_ice
0/100
school_np missforest
0/100
school_np miwae
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/school_np/natural-partition_mnar2-heter/miwae/fedprox/3/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/school_np/natural-partition_mnar2-heter/miwae/fedprox/4/nn/eval_results.json'
2/100
school_np gain
0/100
hhip_np simple
0/100
hhip_np em
0/100
hhip_np linear_ice
0/100
hhip_np missforest
0/100
hhip_np miwae
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/hhip_np/natural-partition_mar-heter/miwae/local/1/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/hhip_np/natural-partition_mar-heter/miwae/local/2/nn/eval_results.json

In [15]:
result_imp, unfinished = load_ret(
    ['vehicle_np', 'school_np', 'hhip_np'],
    ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain'], 
    met = 'imp'
)

vehicle_np simple
0/100
vehicle_np em
0/100
vehicle_np linear_ice
0/100
vehicle_np missforest
0/100
vehicle_np miwae
0/100
vehicle_np gain
0/100
school_np simple
0/100
school_np em
0/100
school_np linear_ice
0/100
school_np missforest
0/100
school_np miwae
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/school_np/natural-partition_mnar2-heter/miwae/fedprox/3/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/school_np/natural-partition_mnar2-heter/miwae/fedprox/4/nn/eval_results.json'
2/100
school_np gain
0/100
hhip_np simple
0/100
hhip_np em
0/100
hhip_np linear_ice
0/100
hhip_np missforest
0/100
hhip_np miwae
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/hhip_np/natural-partition_mar-heter/miwae/local/1/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/hhip_np/natural-partition_mar-heter/miwae/local/2/nn/eval_results.json

In [16]:
df_pred = pd.DataFrame(result_pred)
df_pred = df_pred[df_pred['model'] == 'nn']
df_pred = df_pred.drop(columns=['model'])
df_pred = df_pred[df_pred['client_id'] != 10].reset_index(drop=True)
df_pred = df_pred.replace({'fedtree': 'fedavg'})

dataset_metric = {
    'vehicle_np': 'accuracy',
    'school_np': 'mae',
    'hhip_np': 'mae',
}

dataset_task = {
    'vehicle_np': 'clf',
    'school_np': 'reg',
    'hhip_np': 'reg',
}

df_pred['pred'] = df_pred.apply(lambda x: x[dataset_metric[x['dataset']]], axis=1)
df_pred = df_pred.drop(['accuracy', 'f1', 'auc', 'prc', 'mse', 'mae', 'r2', 'msle'], axis=1)
df_pred['task_type'] = df_pred['dataset'].map(dataset_task)
df_pred.columns = ['dataset', 'partition', 'imp', 'fed', 'missing', 'round_id', 'client_id', 'pred', 'task_type']
df_pred.head()

,dataset,partition,imp,fed,missing,round_id,client_id,pred,task_type
0,vehicle_np,natural-partition,simple,local,mcar,0,0,0.969828,clf
1,vehicle_np,natural-partition,simple,local,mcar,0,1,0.919118,clf
2,vehicle_np,natural-partition,simple,local,mcar,0,2,0.934307,clf
3,vehicle_np,natural-partition,simple,local,mcar,0,3,0.976744,clf
4,vehicle_np,natural-partition,simple,local,mcar,0,4,0.943548,clf


In [17]:
df_imp = pd.DataFrame(result_imp)
df_imp = df_imp[df_imp['client_id'] != 10].reset_index(drop=True)
df_imp = df_imp.drop(columns=['model'])
df_imp = df_imp.replace({'fedtree': 'fedavg'})
df_imp.columns = ['dataset', 'partition', 'imp', 'fed', 'missing', 'round_id', 'client_id', 'rmse', 'sliced-ws']
df_imp.head()

,dataset,partition,imp,fed,missing,round_id,client_id,rmse,sliced-ws
0,vehicle_np,natural-partition,simple,local,mcar,0,0,0.198717,0.059457
1,vehicle_np,natural-partition,simple,local,mcar,0,1,0.193908,0.057849
2,vehicle_np,natural-partition,simple,local,mcar,0,2,0.195987,0.051599
3,vehicle_np,natural-partition,simple,local,mcar,0,3,0.187817,0.054968
4,vehicle_np,natural-partition,simple,local,mcar,0,4,0.190421,0.058600


In [18]:
def agg_client(df, method = 'mean', metric = 'pred'):
    df_agg = df.groupby(['dataset', 'partition', 'missing', 'imp', 'fed', 'client_id'], observed = False).agg(
        {metric: method}).reset_index()
    return df_agg

def agg_client_round(df, method = 'mean', metric = 'pred'):
    df_agg = df.groupby(['dataset', 'partition', 'missing', 'imp', 'fed', 'round_id'], observed = False).agg(
        {metric: method}).reset_index()
    return df_agg

def calculate_improvement(df, metric = 'pred'):
    
    local_baseline = df[df['fed'] == 'local'].copy()
    local_baseline.rename(columns={metric: f'{metric}_local'}, inplace=True)
    keys = ['dataset', 'partition', 'missing', 'imp', 'client_id', 'round_id']
    df = df.merge(local_baseline[keys + [f'{metric}_local']], on=keys, how='left')
    del local_baseline
    
    if metric == 'pred':
    
        df['pred_diff'] = df.apply(
            lambda x: (x['pred'] - x['pred_local']) if x['task_type'] == 'clf' else -(x['pred'] - x['pred_local']), axis=1
        )
        
        # df['pred_improv'] = df.apply(
        #     lambda x: (x['pred_diff'] / (x['pred_local'] + 1e-5)) * 100 if x['task_type'] == 'reg' else x['pred_diff']*100, axis=1
        # )
        df['pred_improv'] = df.apply(
             lambda x: (x['pred_diff'] / (x['pred_local'] + 1e-5)) * 100, axis=1
        )
    else:
        df[f'{metric}_diff'] = df.apply(
            lambda x: -(x[metric] - x[f'{metric}_local']), axis=1
        )
        df[f'{metric}_improv'] = df.apply(
            lambda x: (x[f'{metric}_diff'] / (x[f'{metric}_local'] + 1e-5)) * 100, axis=1
        )
    
    df = df[df['fed'] != 'local']
    #final_df = df[['dataset', 'partition', 'missing', 'imp', 'fed', 'client_id', 'round_id', 'rmse_diff', 'rmse_improv', 'ws_diff', 'ws_improv']]
    
    return df

def agg_client_improv(df, method = 'mean', metric = 'improv', m = 'pred'):
    print(df.columns)
    df['fed'] = df['fed'].astype(str)
    df_agg = df.groupby(['dataset', 'partition', 'missing', 'imp', 'fed', 'client_id'], observed = False).agg(
        {f'{m}_{metric}': method}).reset_index()
    return df_agg

def agg_client_improv2(df, method = 'mean', metric = 'improv', m = 'pred'):
    print(df.columns)
    df['fed'] = df['fed'].astype(str)
    df_agg = df.groupby(['dataset', 'partition', 'missing', 'imp', 'fed', 'round_id'], observed = False).agg(
        {f'{m}_{metric}': method}).reset_index()
    return df_agg

def agg_client_improv_std(df, method = 'std', metric = 'improv', m = 'pred'):
    print(df.columns)
    df['fed'] = df['fed'].astype(str)
    df_agg = df.groupby(['dataset', 'partition', 'missing', 'imp', 'fed', 'round_id'], observed = False).agg(
        {f'{m}_{metric}': method}).reset_index()
    return df_agg

df_pred_improv = calculate_improvement(df_pred)
df_imp_improv = calculate_improvement(df_imp, metric = 'rmse')
df_imp_improv_ws = calculate_improvement(df_imp, metric = 'sliced-ws')

In [24]:
# plotting mapping
dataset_dp = {
    'vehicle_np': ['natural-partition'],
    'school_np': ['natural-partition'],
    'hhip_np': ['natural-partition'],
}

scenario_mapping = {
    'mcar': 'MCAR',
    'mar-homog': 'MAR(HG)',
    'mnar2-homog': 'MNAR(HG)',
    'mnar2-heter': 'MNAR(HT)',
    'mar-heter': 'MAR(HT)',
}

dp_mapping = {
    'natural-partition': 'NP',
}

dataset_mapping = {
    'vehicle_np': 'VHCL',
    'school_np': 'SCHL',
    'hhip_np': 'HHNP',
}

method_mapping = {
    'simple': 'Simple',
    'em': 'EM',
    'linear_ice': 'Linear-ICE',
    'missforest': 'MissForest',
    'miwae': 'MIWAE',
    'gain': 'GAIN',
}

dataset_metric_mapping = {
    'vehicle_np': 'ACCU',
    'school_np': 'MAE',
    'hhip_np': 'MAE',
}

In [27]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
# plt.rcParams['text.usetex'] = False
plt.rcParams.update({'figure.dpi': '600'})
df_imp_improv_avg = agg_client_improv(df_imp_improv, method = 'median', metric = 'improv', m = 'rmse')
df_improv = df_imp_improv_avg
datasets = ['vehicle_np', 'school_np', 'hhip_np']
methods = ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain']
partition = 'natural-partition'
fed = 'fedavg'
scenarios = ['mcar', 'mar-homog', 'mnar2-homog', 'mar-heter', 'mnar2-heter']
metric = 'rmse_improv'

fig, axes = plt.subplots(1, len(datasets), figsize=(10*len(datasets), 5), squeeze=False)
bin_width = 1.0
interval_width = 1.2
lw = 1
fontsize = 20
edgecolor = 'black'

# for each dataset
row_idx = 0
for col_idx, dataset in enumerate(datasets):
    print(dataset)
    result_avg = df_improv[df_improv['dataset'] == dataset]
    print(result_avg)
    result_avg['imp'] = pd.Categorical(result_avg['imp'], categories=methods, ordered=True)
    
    # for each missing scenario
    for i, scenario in enumerate(scenarios):
        filtered_df = result_avg[
            (result_avg['missing'] == scenario) & (result_avg['fed'] == fed)
        ]
        print(filtered_df.shape)
        # for each imputation methods
        bar_heights = []
        bar_errors = []
        for j in range(len(methods)):
            d = filtered_df[filtered_df['imp'] == methods[j]][metric].values
            bar_heights.append(np.median(d))
            bar_errors.append(d.std())
            print(scenario, methods[j], d.mean(), d.std())
            
        bar_positions = [i*interval_width + i*len(methods)*bin_width + j*bin_width for j in range(len(methods))]
        bp = axes[row_idx, col_idx].bar(
            x = bar_positions, height = bar_heights, width = bin_width, linewidth=[lw]*len(methods),
            ecolor=edgecolor, edgecolor = edgecolor, color = colormaps['Paired'].colors[:len(methods)], yerr = bar_errors,
            error_kw = dict(lw=0.7, capsize=0.7, capthick=0.7)
            #hatch = [None, None, None, None,  'x', 'x']
            #color = ['#1e81b0', '#abdbe3', '#7f7f7f', '#d62728'], #hatch = ['\\', 'x', '-',  '/']
        )
        
        # bar labels and positions
        n_groups = len(scenarios)
        n_bars = len(methods)
        x0 = 0 - bin_width/2 - interval_width
        x1 = (n_groups-1)*interval_width + (n_groups-1)*n_bars*bin_width + (n_bars-1)*bin_width + bin_width/2 + interval_width
        axes[row_idx, col_idx].set_xlim([x0, x1])
        axes[row_idx, col_idx].set_xticks([i*interval_width + i*n_bars*bin_width + 1.5*bin_width for i in range(n_groups)])
        axes[row_idx, col_idx].set_xticklabels([scenario_mapping[item] for item in scenarios], fontsize=fontsize, rotation=45) 
        # axes[row_idx, col_idx].set_xlabel('')
        axes[row_idx, 0].set_ylabel('RMSE', fontsize=fontsize)
        # axes[row_idx, col_idx].tick_params(axis='y', which='major', labelsize=fontsize - 2)
        # axes[row_idx, col_idx].avlines = axes[row_idx, col_idx].axhline(y=0, color='black', linewidth=1)
        # axes[row_idx, col_idx].spines['bottom'].set_color('grey')
        # axes[row_idx, col_idx].spines['top'].set_color('grey')
        # axes[row_idx, col_idx].spines['left'].set_color('grey')
        # axes[row_idx, col_idx].spines['right'].set_color('grey')
        
        # title - dataset
    axes[0, col_idx].set_title(dataset, fontsize=fontsize)
    
    legend = plt.legend(
        [bp[i] for i in range(len(methods))], 
        [method_mapping[m] for m in methods], 
        loc='lower center', bbox_to_anchor=(-0.2, -0.5), 
        ncol=6, fontsize = fontsize, frameon=False
    )
    
    for text in legend.get_texts():
        text.set_fontsize(fontsize-1)
        text.set_fontweight('bold')

plt.show()

Index(['dataset', 'partition', 'imp', 'fed', 'missing', 'round_id',
       'client_id', 'rmse', 'sliced-ws', 'rmse_local', 'rmse_diff',
       'rmse_improv'],
      dtype='object')
vehicle_np
         dataset          partition      missing     imp     fed  client_id  \
7400  vehicle_np  natural-partition    mar-heter      em  fedavg          0   
7401  vehicle_np  natural-partition    mar-heter      em  fedavg          1   
7402  vehicle_np  natural-partition    mar-heter      em  fedavg          2   
7403  vehicle_np  natural-partition    mar-heter      em  fedavg          3   
7404  vehicle_np  natural-partition    mar-heter      em  fedavg          4   
...          ...                ...          ...     ...     ...        ...   
8495  vehicle_np  natural-partition  mnar2-homog  simple  fedavg         18   
8496  vehicle_np  natural-partition  mnar2-homog  simple  fedavg         19   
8497  vehicle_np  natural-partition  mnar2-homog  simple  fedavg         20   
8498  vehicle_np  

In [25]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
# plt.rcParams['text.usetex'] = False
plt.rcParams.update({'figure.dpi': '600'})
df_imp_improv_avg = agg_client_improv2(df_imp_improv_ws, method = 'median', metric = 'improv', m = 'sliced-ws')
df_improv = df_imp_improv_avg
datasets = ['vehicle_np', 'school_np', 'hhip_np']
methods = ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain']
partition = 'natural-partition'
fed = 'fedavg'
scenarios = ['mcar', 'mar-homog', 'mnar2-homog', 'mar-heter', 'mnar2-heter']
metric = 'sliced-ws_improv'

fig, axes = plt.subplots(1, len(datasets), figsize=(10*len(datasets), 5), squeeze=False)
bin_width = 1.0
interval_width = 1.2
lw = 1
fontsize = 20
edgecolor = 'black'

# for each dataset
row_idx = 0
for col_idx, dataset in enumerate(datasets):
    print(dataset)
    result_avg = df_improv[df_improv['dataset'] == dataset]
    result_avg['imp'] = pd.Categorical(result_avg['imp'], categories=methods, ordered=True)
    
    # for each missing scenario
    for i, scenario in enumerate(scenarios):
        filtered_df = result_avg[
            (result_avg['missing'] == scenario) & (result_avg['fed'] == fed)
        ]
        print(filtered_df.shape)
        # for each imputation methods
        bar_heights = []
        bar_errors = []
        for j in range(len(methods)):
            d = filtered_df[filtered_df['imp'] == methods[j]][metric].values
            bar_heights.append(np.median(d))
            bar_errors.append(d.std())
            print(scenario, methods[j], d.mean(), d.std())
            
        bar_positions = [i*interval_width + i*len(methods)*bin_width + j*bin_width for j in range(len(methods))]
        bp = axes[row_idx, col_idx].bar(
            x = bar_positions, height = bar_heights, width = bin_width, linewidth=[lw]*len(methods),
            ecolor=edgecolor, edgecolor = edgecolor, color = colormaps['Paired'].colors[:len(methods)], yerr = bar_errors,
            error_kw = dict(lw=0.7, capsize=0.7, capthick=0.7)
            #hatch = [None, None, None, None,  'x', 'x']
            #color = ['#1e81b0', '#abdbe3', '#7f7f7f', '#d62728'], #hatch = ['\\', 'x', '-',  '/']
        )
        
        # bar labels and positions
        n_groups = len(scenarios)
        n_bars = len(methods)
        x0 = 0 - bin_width/2 - interval_width
        x1 = (n_groups-1)*interval_width + (n_groups-1)*n_bars*bin_width + (n_bars-1)*bin_width + bin_width/2 + interval_width
        axes[row_idx, col_idx].set_xlim([x0, x1])
        axes[row_idx, col_idx].set_xticks([i*interval_width + i*n_bars*bin_width + 1.5*bin_width for i in range(n_groups)])
        axes[row_idx, col_idx].set_xticklabels([scenario_mapping[item] for item in scenarios], fontsize=fontsize, rotation=45) 
        # axes[row_idx, col_idx].set_xlabel('')
        axes[row_idx, 0].set_ylabel('sliced-ws', fontsize=fontsize)
        # axes[row_idx, col_idx].tick_params(axis='y', which='major', labelsize=fontsize - 2)
        # axes[row_idx, col_idx].avlines = axes[row_idx, col_idx].axhline(y=0, color='black', linewidth=1)
        # axes[row_idx, col_idx].spines['bottom'].set_color('grey')
        # axes[row_idx, col_idx].spines['top'].set_color('grey')
        # axes[row_idx, col_idx].spines['left'].set_color('grey')
        # axes[row_idx, col_idx].spines['right'].set_color('grey')
        
        # title - dataset
    axes[0, col_idx].set_title(dataset, fontsize=fontsize)
    
    legend = plt.legend(
        [bp[i] for i in range(len(methods))], 
        [method_mapping[m] for m in methods], 
        loc='lower center', bbox_to_anchor=(-0.2, -0.5), 
        ncol=6, fontsize = fontsize, frameon=False
    )
    
    for text in legend.get_texts():
        text.set_fontsize(fontsize-1)
        text.set_fontweight('bold')

plt.show()

Index(['dataset', 'partition', 'imp', 'fed', 'missing', 'round_id',
       'client_id', 'rmse', 'sliced-ws', 'sliced-ws_local', 'sliced-ws_diff',
       'sliced-ws_improv'],
      dtype='object')
vehicle_np
(50, 7)
mcar simple -3.1051642112117595 0.46126688114003433
mcar em -5.107186755201555 0.2953185658581864
mcar linear_ice -13.676909879024226 1.2933891399388362
mcar missforest -24.761238304789927 2.607985369018672
mcar miwae -9.94257460496871 0.6553195216144565
mcar gain -12.02073258873814 2.604866199363439
(50, 7)
mar-homog simple 0.3892095805736987 1.4480221477776714
mar-homog em -7.275048482800278 3.5873126002385733
mar-homog linear_ice -6.111591548213658 5.107844311913661
mar-homog missforest -19.638077953808867 4.869973297442666
mar-homog miwae -20.269988828894668 4.527882593974991
mar-homog gain 1.4767209224797793 12.452036576347874
(50, 7)
mnar2-homog simple -0.7240207474239553 0.326548724860649
mnar2-homog em -1.6011959340407498 0.31651833467933604
mnar2-homog linear_ice -0

In [23]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
# plt.rcParams['text.usetex'] = False
plt.rcParams.update({'figure.dpi': '600'})
df_pred_improv_avg = agg_client_improv2(df_pred_improv, method = 'median', metric = 'improv', m = 'pred')
df_improv = df_pred_improv_avg
datasets = ['vehicle_np', 'school_np', 'hhip_np']
methods = ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain']
partition = 'natural-partition'
fed = 'fedavg'
scenarios = ['mcar', 'mar-homog', 'mnar2-homog', 'mar-heter', 'mnar2-heter']
metric = 'pred_improv'

fig, axes = plt.subplots(1, len(datasets), figsize=(10*len(datasets), 5), squeeze=False)
bin_width = 1.0
interval_width = 1.2
lw = 1
fontsize = 20
edgecolor = 'black'

# for each dataset
row_idx = 0
for col_idx, dataset in enumerate(datasets):
    print(dataset)
    result_avg = df_improv[df_improv['dataset'] == dataset]
    result_avg['imp'] = pd.Categorical(result_avg['imp'], categories=methods, ordered=True)
    
    # for each missing scenario
    for i, scenario in enumerate(scenarios):
        filtered_df = result_avg[
            (result_avg['missing'] == scenario) & (result_avg['fed'] == fed)
        ]
        print(filtered_df.shape)
        # for each imputation methods
        bar_heights = []
        bar_errors = []
        for j in range(len(methods)):
            d = filtered_df[filtered_df['imp'] == methods[j]][metric].values
            bar_heights.append(np.median(d))
            bar_errors.append(d.std())
            print(scenario, methods[j], d.mean(), d.std())
            
        bar_positions = [i*interval_width + i*len(methods)*bin_width + j*bin_width for j in range(len(methods))]
        bp = axes[row_idx, col_idx].bar(
            x = bar_positions, height = bar_heights, width = bin_width, linewidth=[lw]*len(methods),
            ecolor=edgecolor, edgecolor = edgecolor, color = colormaps['Paired'].colors[:len(methods)], yerr = bar_errors,
            error_kw = dict(lw=0.7, capsize=0.7, capthick=0.7)
            #hatch = [None, None, None, None,  'x', 'x']
            #color = ['#1e81b0', '#abdbe3', '#7f7f7f', '#d62728'], #hatch = ['\\', 'x', '-',  '/']
        )
        
        # bar labels and positions
        n_groups = len(scenarios)
        n_bars = len(methods)
        x0 = 0 - bin_width/2 - interval_width
        x1 = (n_groups-1)*interval_width + (n_groups-1)*n_bars*bin_width + (n_bars-1)*bin_width + bin_width/2 + interval_width
        axes[row_idx, col_idx].set_xlim([x0, x1])
        axes[row_idx, col_idx].set_xticks([i*interval_width + i*n_bars*bin_width + 1.5*bin_width for i in range(n_groups)])
        axes[row_idx, col_idx].set_xticklabels([scenario_mapping[item] for item in scenarios], fontsize=fontsize, rotation=45) 
        # axes[row_idx, col_idx].set_xlabel('')
        axes[row_idx, 0].set_ylabel('PRED', fontsize=fontsize)
        # axes[row_idx, col_idx].tick_params(axis='y', which='major', labelsize=fontsize - 2)
        # axes[row_idx, col_idx].avlines = axes[row_idx, col_idx].axhline(y=0, color='black', linewidth=1)
        # axes[row_idx, col_idx].spines['bottom'].set_color('grey')
        # axes[row_idx, col_idx].spines['top'].set_color('grey')
        # axes[row_idx, col_idx].spines['left'].set_color('grey')
        # axes[row_idx, col_idx].spines['right'].set_color('grey')
        
        # title - dataset
    axes[0, col_idx].set_title(dataset, fontsize=fontsize)
    
    legend = plt.legend(
        [bp[i] for i in range(len(methods))], 
        [method_mapping[m] for m in methods], 
        loc='lower center', bbox_to_anchor=(-0.2, -0.5), 
        ncol=6, fontsize = fontsize, frameon=False
    )
    
    for text in legend.get_texts():
        text.set_fontsize(fontsize-1)
        text.set_fontweight('bold')

plt.show()

Index(['dataset', 'partition', 'imp', 'fed', 'missing', 'round_id',
       'client_id', 'pred', 'task_type', 'pred_local', 'pred_diff',
       'pred_improv'],
      dtype='object')
vehicle_np
(50, 7)
mcar simple -0.19162390292620243 0.381485059875454
mcar em -0.02652929190821477 0.24915168582678443
mcar linear_ice 0.0836512429227155 0.3007907890706635
mcar missforest 0.11924857607932537 0.25008069679680317
mcar miwae 0.1613967663523292 0.21775459257410762
mcar gain -0.04566163967021138 0.09132327934042277
(50, 7)
mar-homog simple -0.04560839870679467 0.44146392823593295
mar-homog em 0.02479562428261221 0.2831817056046165
mar-homog linear_ice -0.173223591108629 0.2809244886397954
mar-homog missforest 0.13077606653399515 0.47196596076790326
mar-homog miwae 0.27035494778735814 0.33551978992407705
mar-homog gain -0.11649300807852905 0.12729464297963972
(50, 7)
mnar2-homog simple 0.10346098805621953 0.38246588447401514
mnar2-homog em 0.045654794623866776 0.44605347869263845
mnar2-homog line